Para la implementeacion de la Api se tiene en cuenta la siguiente estructura del proyecto 

En este proyecto, se está desarrollando un flujo completo de despliegue de un modelo de clasificación como un servicio web utilizando FastAPI. Este proceso permite exponer un modelo de Machine Learning entrenado para que pueda ser consumido por aplicaciones externas a través de una API REST

📂 fastapi-ml-deployment
│── 📂 app
│   ├── model.pkl                 # Modelo entrenado guardado
│   ├── app.py                     # Código de la API con FastAPI
│── notebook.ipynb                 # Notebook de Google Colab documentando el proceso
│── requirements.txt                # Librerías necesarias
│── README.md                       # Instrucciones del proyecto

![alt text](image.png)

Este repositorio se encuentra subido dentro de Github

** Se procede a realizar el entrenamiento del modelo **

In [ ]:
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar dataset (reemplaza con tu dataset real si es necesario)
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv", header=None)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo de clasificación
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Evaluar modelo
y_pred = modelo.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Guardar modelo en un archivo
with open("model.pkl", "wb") as f:
    pickle.dump(modelo, f)

![alt text](image-1.png)

Crear la API con FastAPI Logging y Monitoreo

In [ ]:
import os
from fastapi import FastAPI
import pickle
import numpy as np
from pydantic import BaseModel
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)

# Obtener la ruta absoluta del directorio actual
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

# Cargar el modelo desde la carpeta correcta
model_path = os.path.join(BASE_DIR, "model.pkl")

with open(model_path, "rb") as f:
    modelo = pickle.load(f)

# Inicializar FastAPI
app = FastAPI()

# Definir esquema de entrada
class InputData(BaseModel):
    features: list

# Endpoint para predicción
@app.post("/predict/")
def predict(data: InputData):
    X_new = np.array(data.features).reshape(1, -1)
    prediction = modelo.predict(X_new)
    
    # Registrar la solicitud y respuesta
    logging.info(f"Entrada: {data.features}, Predicción: {prediction[0]}")
    
    return {"prediction": int(prediction[0])}

![alt text](image-2.png)

Ejecutar la API Localmente

uvicorn app.app:app --reload

![alt text](image-3.png)

documentación interactiva en

In [ ]:
http://127.0.0.1:8000/docs

![alt text](image-4.png)

Ejecucion en Postman 

![alt text](image-5.png)

Lista de Dependencias 

![alt text](image-6.png)

Conclusión 

Este proyecto demuestra cómo desplegar un modelo de Machine Learning como un servicio web, facilitando su integración en aplicaciones externas sin necesidad de acceder al código original del modelo.

Beneficios clave:

Permite realizar predicciones en tiempo real mediante solicitudes HTTP.

Facilita la integración con aplicaciones web o móviles.

Mejora la reutilización del modelo sin necesidad de reentrenarlo constantemente.

Implementa buenas prácticas de monitoreo y registro de eventos.

El despliegue de modelos con FastAPI es una solución eficiente y escalable para hacer accesibles modelos de Machine Learning en entornos productivos, permitiendo su uso desde cualquier sistema que pueda realizar solicitudes HTTP